<a href="https://colab.research.google.com/github/param-b/Enhanced-Controllable-Text-Summarization/blob/main/Enhanced_Controllable_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [ ]:
! pip install keybert
! pip install datasets
! pip install summarizers
! pip install rouge_score
! pip install -U pip setuptools wheel
! pip install -U spacy
! pip install SPARQLWrapper 
! python -m spacy download en_core_web_md

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 232 kB 24.0 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
     |████████████████████████████████| 4.0 MB 91.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 40.2 MB/s 
     |████████████████████████████████| 895 kB 53.3 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=e692140be7f519cbffad43cdac494b0a276c562e07d097b0f921cebbf7901f37
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=c2ef8e4a23f487cc297a7f90d8d65deaab727932c99e9e8808375ae7a53eb5ef
  Stored in directory: /root/.cache/pip/whee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 KB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.3/653.3 KB 51.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempt

# Import libraries

In [ ]:
from keybert import KeyBERT
from datasets import load_dataset
from datasets import load_metric
from summarizers import Summarizers

import spacy
import en_core_web_sm
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import gensim.downloader
from SPARQLWrapper import SPARQLWrapper2

import numpy as np
import tqdm
import re
import copy 
from itertools import chain, product


/usr/local/lib/python3.7/dist-packages/spacy/util.py:841: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.2.4,<3.3.0
  warnings.warn(warn_msg)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Class Instances

In [ ]:
dataset = load_dataset("cnn_dailymail", '3.0.0', split="test")
summ = Summarizers('normal', device='cuda')
rouge = load_metric("rouge")
kw_model = KeyBERT()

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
nlp = spacy.load("en_core_web_md")
# nlp = en_core_web_sm.load()

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[==================================================] 100.0% 128.1/128.1MB downloaded


# OOV Model

# Function

In [ ]:
def query_dbpedia2(subject):
    graph = {}
    sparql = SPARQLWrapper2("http://dbpedia.org/sparql")
    
    input_query = lambda p,s,prefix:"""
        PREFIX dbp:  <http://dbpedia.org/resource/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?%s
        WHERE {
            dbp:%s  %s:%s ?%s
            FILTER(LANG(?%s)='en')
        }
        """%(p, s, prefix, p, p, p)

    # return dbo:abstract
    values = []
    input_query1 = input_query('abstract', subject, 'dbo')
    sparql.setQuery(input_query1)
    for result in sparql.query().bindings:
        values.append(result['abstract'].value)
    graph['abstract'] = values

    
    # return rdfs:comment
    values = []
    input_query0 = input_query('comment', subject, 'rdfs')
    sparql.setQuery(input_query0)
    for result in sparql.query().bindings:
        values.append(result['comment'].value)
    graph['comment'] = values
    
    
    # return rdf:type
    rx = re.compile(r'(?<=[a-z])(?=[A-Z])')
    input_query2 = """
        PREFIX dbp:  <http://dbpedia.org/resource/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?type
        WHERE {
            dbp:%s  rdf:type ?type
        }
        """%(subject)
    sparql.setQuery(input_query2)

    keywords = []
    rx = re.compile(r'(?<=[a-z])(?=[A-Z])')
    for result in sparql.query().bindings:
        if result['type'].value.find('dbpedia.org/ontology')>0:
            keyword = result['type'].value.split('/')[-1]
            keyword = rx.sub(' ', keyword)
            keywords.append(keyword.replace("_"," "))
        if result['type'].value.find('umbel.org/umbel/rc')>0:
            keyword = result['type'].value.split('/')[-1]
            keyword = rx.sub(' ', keyword)
            keywords.append(keyword.replace("_"," "))

    graph['type'] = keywords
    return graph


def query_to_wordbank(subject, kw_model=kw_model):
    graph = query_dbpedia2(subject)
    wordbank = graph_to_wordbank(graph, kw_model)
    wordbank = filter_wordbank(wordbank)
    return wordbank


def filter_wordbank(wordbank, keep_prob=False, threshold=0.25):
    filter_wb = {}
    for keys in wordbank.keys():
        filter_wb[keys] = []
        filter_wb['type'] = wordbank['type']
        if keys != 'type':
            for n in range(len(wordbank[keys])): #
                # print(n)
                kwprob_pair = wordbank[keys][n]
                # print(kwprob_pair)
                if kwprob_pair[1]>threshold:
                    if keep_prob:
                        filter_wb[keys].append(wordbank[keys][n])
                    else:
                        filter_wb[keys].append(wordbank[keys][n][0])
    return filter_wb


def graph_to_wordbank(graph, kw_model=kw_model):
    wordbank = {}
    # extract keywords from abstract
    if graph['abstract'] != []:
        text = graph['abstract'][0]
        keywords = kw_model.extract_keywords(text, 
                                             keyphrase_ngram_range=(1, 1), 
                                             stop_words='english', 
                                             highlight=False, use_mmr=True, diversity=0.2,
                                             top_n=20)

        wordbank['abstract'] = keywords

        # extract keywords from comment
        text = graph['comment'][0]
        keywords = kw_model.extract_keywords(text, 
                                             keyphrase_ngram_range=(1, 1), 
                                             stop_words='english', 
                                             highlight=False, use_mmr=True, diversity=0.1,
                                             top_n=10)
        wordbank['comment'] = keywords

        # extract keywords from type
        wordbank['type'] = list(set(graph['type']))
    return wordbank


# workflow:
def does_it_qualify(token):
    if str(token.pos_).upper() == "VERB":
        return True
    elif str(token.pos_).upper() == "VERB":
        return True
    elif str(token.pos_).upper() == "ADJ":
        return True
    elif str(token.pos_).upper() == "NOUN":
        return True
    return False


def find_similar_word_in_article(token, text):
    sim_words = []
    # Add Definition from Sys Net or somethigng like that maybe
    for word in text:
        try:
            similar = glove_vectors.similarity(
                str(token.text).lower(),
                str(word.text).lower())
            if similar > 0.8:
                sim_words.append(word)
        except:
            print(word + " Not in VOCABULARY ")
    return sim_words


# def find_similar_word_in_article_for_user_pref(token, doc, threshold=0.05):
#     sim_words = []
#     similarities = []
#     word_list = []
#     # Add Definition from Sys Net or somethigng like that maybe
#     for word in doc:
#         if does_it_qualify(nlp(word.lower_)[0]):
#             try:
#                 similar_2 = token.similarity(word)
#                 similar_1 = glove_vectors.similarity(
#                     str(token.text).lower().strip(),
#                     str(word.text).lower().strip()
#                 )
#                 similarity = similar_2*similar_1
                
#             except Exception as e:
#                 #print(str(word) + " Not in VOCABULARY ")
#                 #print(e)
#                 pass
            
#             word_list.append(word)
#             similarities.append(similarity)
#         sim_words = list(np.array(word_list)[list(np.array(similarities)>(np.array(similarities).mean()+threshold))])
#     return sim_words#, word_list, similarities


# For Adjs Noun Etc
def find_similar_word_in_article_for_user_pref(token, text_nlp):
    sim_words = []
    # Add Definition from Sys Net or somethigng like that maybe
    # print('token: ' + token.text)
    for word in text_nlp:
        #print(word)
        if does_it_qualify(word):
            try:
                #print(f'Qualified word in corpus: {word}')
                # print(f"____{word}_______")
                #similar_2 = token.similarity(word)
                #print("Similar2 Score: " + str(similar_2))
                similar = glove_vectors.similarity(
                    str(token.text).lower().strip(),
                    str(word.text).lower().strip())
                #print("Similar Score: " + str(similar))
                similar_3 = -0.5
                try:
                  # WordNet Part Comes Here!
                  # syn1 = wordnet.synsets(str(token.text).lower())
                  # syn2 = wordnet.synsets(str(word.text).lower())
                  # similar_3 = syn1.wup_similarity(syn2)
                  # print ("hello name :  ", syn1.name())
                  # print ("selling name :  ", syn2.name())
                  
                  syns1 = wordnet.synsets(str(token.text).lower())
                  syns2 = wordnet.synsets(str(word.text).lower())
                  # print(syns1)
                  # print(syns2)
                  for sense1, sense2 in product(syns1, syns2):
                    # print(sense1)
                    # print(sense2)
                    d = sense1.wup_similarity(sense2) or 0.0
                    # print(d)
                    similar_3 = max(similar_3, d)
                  #print("Similar3 Score: " + str(similar_3))
                except Exception as e:
                  print(e)
                  print("WordNet Issue")
                  similar_3 = -1.0
                # print(similar)
                # print(similar_2)
                # print(similar_3)
                if similar > 0.75 or similar_3 > 0.6:
                    sim_words.append(word.text)
                    #print("Similar Word: " + word.text)
            except Exception as e:
                #print(str(word) + " Not in VOCABULARY ")
                #print('HERE')
                #print(e)
                pass
    return sim_words


# For Proper Nouns
def proper_nn_qualifies(word, label, kw_model=kw_model):
    split_word = str(word).title().split(" ")
    subject = "_".join(split_word)
    subject = re.sub(r'\'\w+', '', subject)
    new_word = subject
    print(new_word)
    graph = query_dbpedia2(subject)
    wordbank = query_to_wordbank(subject, kw_model)
    soln = []
    # To Create one single list of all the values
    for temp_entry in wordbank.values():
        for temp_temp_entry in temp_entry:
            soln.append(temp_temp_entry)
    print(label)
    print(soln)
    if str(label).upper() == "NORP":
        # Make call to knowledge graph acc to the proper npun type
        return True, new_word, soln
    elif str(label).upper() == "PERSON":
        return True, new_word, soln
    elif str(label).upper() == "FAC":
        return True, new_word, soln
    elif str(label).upper() == "GPE":
        return True, new_word, soln
    elif str(label).upper() == "LOC":
        return True, new_word, soln
    elif str(label).upper() == "PRODUCT":
        return True, new_word, soln
    elif str(label).upper() == "EVENT":
        return True, new_word, soln
    elif str(label).upper() == "WORK_OF_ART":
        return True, new_word, soln
    return False, new_word, []


#similarity matching
def is_similar(similarity, threshold=0.8):
    return similarity >= threshold


def find_matching_func(user_pref_dict, proper_input):
    updated_user_pref_dict = user_pref_dict.copy()
    for user_item in set(updated_user_pref_dict.keys()):
        for user_item_split in user_item.split(" "):
            for proper_name, proper_name_val in proper_input.items():
                for values in proper_name_val:
                    for single_value in values.split(" "):
                        # print(single_value)
                        try:
                            if str(user_item).lower().strip() == str(
                                    single_value).lower().strip():
                                updated_user_pref_dict[user_item].append(
                                    str(proper_name).strip().lower())
                            elif (is_similar(
                                    glove_vectors.similarity(
                                        str(single_value).strip().lower(),
                                        str(user_item_split).strip().lower()))):
                                updated_user_pref_dict[user_item].append(
                                    str(proper_name).strip().lower())
                                # Use wordnet if needed
                        except:
                            pass
    return updated_user_pref_dict
                  
                    
def make_everything_lower(user_list):
    new_list = []
    for entry in user_list:
        new_list.append(str(entry).lower())
    return new_list


def remove_unneccesary_keys(og_user_query, updated_user_pref_dict):
    temp_new_dict = {}
    new_og_user_query = make_everything_lower(og_user_query)
    for key in updated_user_pref_dict.keys():
        if key.lower() in new_og_user_query:
            temp_new_dict[key] = list(set(make_everything_lower(updated_user_pref_dict[key])))
    return temp_new_dict


def find_matching_func(user_pref_dict, proper_input):
    updated_user_pref_dict = user_pref_dict.copy()
    for user_item in set(updated_user_pref_dict.keys()):
        for user_item_split in user_item.split(" "):
            for proper_name, proper_name_val in proper_input.items():
                for values in proper_name_val:
                    for single_value in values.split(" "):
                        # print(single_value)
                        try:
                            if str(user_item).lower().strip() == str(
                                    single_value).lower().strip():
                                updated_user_pref_dict[user_item].append(
                                    str(proper_name).strip().lower())
                            elif (is_similar(
                                    glove_vectors.similarity(
                                        str(single_value).strip().lower(),
                                        str(user_item_split).strip().lower()))):
                                updated_user_pref_dict[user_item].append(
                                    str(proper_name).strip().lower())
                                # Use wordnet if needed
                        except:
                            pass
    return updated_user_pref_dict


def match_proper_noun_dict(word, word_bank_pn, user_pref_dict):
    word_list = word_bank_pn[word]
    #user_list_merged = str(" ".join(user_list)).lower()
    ctr = 0
    for kg in word_list:
        print(kg)
        if str(kg).lower() in user_pref_dict.keys():
            ctr += 1
            user_pref_dict[str(kg).lower()].append(word)
    #ppn_matching[word] = ctr
    return user_pref_dict


# article to wordbank
def get_wordbank(article, og_user_query, kw_model=kw_model):
    # Create Tokens
    doc = nlp(str(article))
    word_bank = {}
    word_bank_pn = {}
    # create the user preference dictionary
    user_pref_dict ={str(user_pref).lower(): [] for user_pref in og_user_query}
    # print(user_pref_dict)

    # Find all similar words in corpus
    for user_pref in og_user_query:
        doc_temp = nlp(user_pref)
        for token in doc_temp:
            sim_words = find_similar_word_in_article_for_user_pref(token, doc)
            user_pref_dict[str(token.text)] = sim_words
            
    # For Proper Nouns create word bank from the article and then match
    for X in doc.ents:
        # print(X.text)
        # print(str(X.label_).upper() == "PERSON")
        isTrue, entity, soln = proper_nn_qualifies(X.text, X.label_, kw_model=kw_model)
        # print(isTrue)
        if (isTrue):
            try:
                word_bank_pn[str(X.text)] = soln
                user_pref_dict = match_proper_noun_dict(str(X.text))
            except:
                pass
    # merge wordbank into preference:
    updated_user_pref_dict = find_matching_func(user_pref_dict, word_bank_pn)
    return updated_user_pref_dict, user_pref_dict, word_bank_pn


def get_wordbank_evalutaion(article, og_user_query, kw_model=kw_model):
    # Create Tokens
    doc = nlp(str(article))
    word_bank = {}
    word_bank_pn = {}
    # create the user preference dictionary
    user_pref_dict ={str(user_pref).lower(): [] for user_pref in og_user_query}
    # print(user_pref_dict)

    # Find all similar words in corpus
    for user_pref in og_user_query:
        doc_temp = nlp(user_pref)
        for token in doc_temp:
            sim_words = find_similar_word_in_article_for_user_pref(token, doc)
            user_pref_dict[str(token.text)] = sim_words
            
    # For Proper Nouns create word bank from the article and then match
    # for X in doc.ents:
    #     # print(X.text)
    #     # print(str(X.label_).upper() == "PERSON")
    #     isTrue, entity, soln = proper_nn_qualifies(X.text, X.label_, kw_model=kw_model)
    #     # print(isTrue)
    #     if (isTrue):
    #         try:
    #             word_bank_pn[str(X.text)] = soln
    #             user_pref_dict = match_proper_noun_dict(str(X.text))
    #         except:
    #             pass
    # # merge wordbank into preference:
    # updated_user_pref_dict = find_matching_func(user_pref_dict, word_bank_pn)
    updated_user_pref_dict = user_pref_dict
    return updated_user_pref_dict


def convert_wordbank_to_user_str(word_bank_dict):
  temp_str = ""
  for key, value in word_bank_dict.items():
    temp_str_now= " " .join(value)
    temp_str = temp_str + temp_str_now
  return temp_str


def find_the_most_similar_word(keyword):
  try:
    synonyms = wordnet.synsets(keyword)
    for synonym in synonyms:
      synonym_word = synonym.name().split('.')[0]
      if (synonym_word.lower() != keyword):
        return synonym_word.lower().replace('_', ' ')
    
      glove = glove_vectors.most_similar(str(keyword), topn = 1)

      if (len(glove) > 0):
        return glove[0][0].lower().replace('_', ' ')
      
    return ''
    
  except Exception as e:
    print(e)
    return ''


def extract_keyword(sentence):
  keyword = kw_model.extract_keywords(sentence, stop_words='english', top_n=1)
  return keyword[0][0].lower()

# Evaluation

In [ ]:
def compute_metrics(ground_truth, ctrl_sum_unchanged, ctrl_sum_changed, oov_changed):
    ctrl_sum_uchanged_result = rouge.compute(predictions=ctrl_sum_unchanged, references=ground_truth)
    ctrl_sum_changed_result = rouge.compute(predictions=ctrl_sum_changed, references=ground_truth)
    oov_result = rouge.compute(predictions=oov_changed, references=ground_truth)

    # return {ctrl_sum: round(v, 4) for k, v in result.items(), oov: None}
    return {'ctrl_sum_uchanged_f1': {key: value.mid.fmeasure * 100 for key, value in ctrl_sum_uchanged_result.items()}, 
            'ctrl_sum_changed_f1': {key: value.mid.fmeasure * 100 for key, value in ctrl_sum_changed_result.items()} ,
            'oov_f1': {key: value.mid.fmeasure * 100 for key, value in oov_result.items()}}

# Actual Test

In [ ]:
sentences, ctrl_summaries_unchanged, ctrl_summaries_changed, oov_summaries_changed= [], [], [], []
acc = 0
num = 1
total_sentence_num = 0
dataset_len = len(dataset)
for data in tqdm.tqdm(dataset):
    sentences_list = data['highlights'].split('.\n') # Ground Truth Summary Sentences
    total_sentence_num += len(sentences_list)
    for sentence in sentences_list:

      # Clean up the sentence
      sentence = sentence.replace(' .', '').strip()
      if (sentence[-1] != '.'):
        sentence += '.'
      
      keyword = extract_keyword(sentence)
      changed_keyword = find_the_most_similar_word(keyword)

      # Disregard Sentence if changed_keyword is empty string or equal to original keyword
      if str(changed_keyword).strip() != '' and str(changed_keyword).lower().strip() != str(keyword).lower().strip():
        wordbank = get_wordbank_evalutaion(sentence, [changed_keyword], kw_model=kw_model)
        updated_keyword = convert_wordbank_to_user_str(wordbank)

        # Disregard Sentence if model cannot find similar word in the knowledge graph
        if updated_keyword != '':
          ctrl_summaries_unchanged.append(str((summ(contents=sentence, query=keyword, question_detection=False)))+"\n")
          ctrl_summaries_changed.append(str((summ(contents=sentence, query=changed_keyword, question_detection=False)))+"\n")
        
          # Our Model
          oov_summaries_changed.append(str(summ(contents=sentence, query=updated_keyword, question_detection=False))+"\n")

          # Rouge expects a newline after each sentence
          sentences.append(str(sentence)+"\n")
        
    # To track the progress
    if acc == dataset_len/10:
      print(f'Progress Report: {int(num * ((acc/dataset_len) * 100))}%')
      acc = 0
      num += 1
      
    acc +=1

 10%|█         | 1149/11490 [31:24<3:32:29,  1.23s/it]

Progress Report: 10%


 20%|██        | 2299/11490 [1:13:16<7:31:15,  2.95s/it]

Progress Report: 20%


 30%|███       | 3448/11490 [1:55:58<4:43:01,  2.11s/it]

...

 80%|████████  | 9193/11490 [5:23:08<1:11:37,  1.87s/it]

Progress Report: 80%


 90%|█████████ | 10342/11490 [6:06:12<47:13,  2.47s/it]

Progress Report: 90%


100%|██████████| 11490/11490 [6:48:40<00:00,  2.13s/it]


In [ ]:
metrics = compute_metrics(ground_truth=sentences, ctrl_sum_changed=ctrl_summaries_changed, ctrl_sum_unchanged=ctrl_summaries_unchanged, oov_changed=oov_summaries_changed)
for key, values in metrics.items():
  print(f'{key}: {values}')

ctrl_sum_uchanged_f1: {'rouge1': 92.7290679056413, 'rouge2': 91.03397922130603, 'rougeL': 92.38905962198632, 'rougeLsum': 92.5001444785213}
ctrl_sum_changed_f1: {'rouge1': 86.87973415821011, 'rouge2': 84.59173517006636, 'rougeL': 86.06145540249483, 'rougeLsum': 86.28728152841259}
oov_f1: {'rouge1': 94.81309903816695, 'rouge2': 93.36058070970677, 'rougeL': 94.37920834700019, 'rougeLsum': 94.6162687518369}


In [ ]:
print(total_sentence_num)
print(len(sentences))
print(len(ctrl_summaries_changed))
print(len(ctrl_summaries_unchanged))
print(len(oov_summaries_changed))

43073
21941
21941
21941
21941
